<a href="https://colab.research.google.com/github/chengsun1/NLPgroup9/blob/main/Hugging_face_nlptown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import transformers
import torch

In [4]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 14.8 MB/s eta 0:00:00


In [2]:
dftr = pd.read_csv('train.csv')


In [3]:
train_df, eval_df = train_test_split(dftr, test_size=0.15, stratify=dftr['label'], random_state=42)

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [5]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

In [6]:
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

In [7]:
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
eval_dataset = eval_dataset.map(tokenize, batched=True, batch_size=len(eval_dataset))

Map:   0%|          | 0/17898 [00:00<?, ? examples/s]

Map:   0%|          | 0/3159 [00:00<?, ? examples/s]

In [8]:
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
eval_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [9]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 10.5 MB/s eta 0:00:00


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [11]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,3.251300
20,2.577900
30,1.672500
40,1.329400
50,1.277200
60,1.160800
70,1.132600
80,1.080400
90,1.092300
100,1.049600


TrainOutput(global_step=3357, training_loss=0.7534574618607999, metrics={'train_runtime': 1254.9837, 'train_samples_per_second': 42.785, 'train_steps_per_second': 2.675, 'total_flos': 1.4127865541462016e+16, 'train_loss': 0.7534574618607999, 'epoch': 3.0})

In [14]:
from sklearn.metrics import f1_score
import numpy as np
predictions, labels, _ = trainer.predict(eval_dataset)
preds = np.argmax(predictions, axis=1)


In [15]:
f1 = f1_score(labels, preds, average='weighted')
print(f'F1 Score: {f1}')

F1 Score: 0.6237327276490481


In [16]:
pred = pd.read_csv('test.csv')

In [22]:
pred_dataset = Dataset.from_pandas(pred)
pred_dataset = pred_dataset.map(tokenize, batched=True, batch_size=len(pred_dataset))

Map:   0%|          | 0/8943 [00:00<?, ? examples/s]

In [26]:
pred_encoded = tokenizer(pred_dataset['text'], truncation=True, padding=True)


In [30]:
pred_dataset = Dataset.from_dict(pred_encoded)
predictions, _, _ = trainer.predict(pred_dataset)
preds = np.argmax(predictions, axis =1)

In [18]:
output = pd.read_csv('submission.csv')

In [33]:
output['pred_label']=preds

In [37]:
output.to_csv('submission_hugging_face.csv', index = False)